In [21]:
# El primer paso es importar pandas y cargar nuestro dataset,por facilidad se coloca el dataset en la misma carpeta de nuestro código

import pandas as pd

In [22]:
data_original = pd.read_csv("c:\Users\USER\Desktop\MAESTRÍA - BIG DATA\MATERIAS\3.- MODELO Y APRENDIZAJES\EXAMEN\wdbc.csv")

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (2711503244.py, line 1)

In [ ]:
data_original.head(5)

: 

In [ ]:
data_original.info()

: 

In [ ]:
# la calidad de vino tiene un valor minimo de 3 y maximo de 8 esta caracteristica del dataset será importante para la predicción
# se recomdaría utilizar un dataset que contenga vinos con todas las calidades posibles
data_original.describe()

: 

In [ ]:
#creamos una copia para empezar a limpiar los datos
data_work = data_original.copy()

: 

In [ ]:
#convertimos la columna aged date a fecha
data_work["TIME"] = pd.to_datetime(data_work["TIME"]).dt.date

: 

In [ ]:
wine_work["TIME"]

: 

In [ ]:
#localizamos los duplicados
duplicates = wine_work.duplicated('index', keep = False)
wine_work[duplicates]

: 

In [ ]:
#eliminamos los duplicados y conservamos la primera aparación
wine_work = wine_work.drop_duplicates(subset = "index", keep = "first")

: 

In [ ]:
#confirmamos que ya no temeos valores duplicados
duplicates = wine_work.duplicated('index', keep = False)
wine_work[duplicates]

: 

In [ ]:
# localizamos los valores null
null_columns= wine_work.columns[wine_work.isnull().any()]
wine_work[null_columns].isnull().sum()

: 

In [ ]:
wine_work[wine_work["LOCATION"].isnull()]

: 

In [ ]:
# decidimos rellenar los duplicados con el país Ecuador
wine_work["LOCATION"].fillna("Ecuador", inplace = True)

: 

In [ ]:
#comprobamos que ya no hat duplicados
null_columns2= wine_work.columns[wine_work.isnull().any()]
wine_work[null_columns].isnull().sum()

: 

In [ ]:
#cmabiamos el indice por el id del estudiante 
wine_work.set_index("index", inplace = True)


: 

In [ ]:
# creamos una columna con el nombre de quality label que se basa en la columna quality
# con un función lambda creamos una condición: si el vino tiene una calificación mayor a 5 es "Good", si es menor es "Bad"
# se mejoro la predicción cuando se cambio la calidad a considerar Good desde 5, eso se explica porque la mayor distribución
# se encuentra a partir de la calificación 5
wine_work["student label"] = wine_work["SIZE-CM"].apply(lambda x: "BAJO" if x <= 155  else "ALTO")

: 

In [ ]:
wine_work.head(10)

: 

In [ ]:
import numpy as np
piechart = np.array(wine_work["SIZE-CM"])
np.info(piechart)

: 

In [ ]:
#la mayoría de valores soon mayores o iguales a 5 esto limitará la predicción  de vinos Bad
import matplotlib.pyplot as plt

plt.hist(piechart)
plt.show() 

: 

In [ ]:
#importamos sklearn y creamos nuestras variables para el análisis
# la variable X contiene las caracteristicas del vino desde fixed acidity hasta alcohol
#la variable y contiene los datos de la columna quality label

from sklearn.model_selection import train_test_split
X= wine_work.copy().iloc[:,3:-1]
y= wine_work["student label"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=45)

: 

In [ ]:
X.info()

: 

In [ ]:
#nos cercioramos que los datos de nuestro DataFrame son númericos
X = X.astype(float)

: 

In [ ]:
y.head()

: 

In [ ]:
# importamos la herramienta para desarrollar el árbol de decisión
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(max_depth=5)
clf = clf.fit(X_train, y_train)

: 

In [ ]:
X_test

: 

In [ ]:
#ejecutamos la predicción
predictions = clf.predict(X_test)
predictions

: 

In [ ]:
#importamos métricas importantes para interpretarlas
#accuray score nos permite conocer el valor porcentual de aciertos totales en este caso 95%

from sklearn.metrics import accuracy_score
accuracy_score(y_test, predictions)

: 

In [ ]:
#la matriz de confusión nos enseña cuantas veces una clase fue asiganada correctamente en este caso 501 veces Good fue asignado correctamente
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions,labels = ("BAJO", "ALTO"))

: 

In [ ]:
#precision score es el porcentaje de correctos positivos por cada clase, en este caso 96% de precision para Good
from sklearn.metrics import precision_score
precision_score(y_test, predictions, pos_label = "BAJO")

: 

In [ ]:
#precision score para vinos Bad es solo 42% esto porque no hay demasiados vinos con calificación menor a 5 en el dataset
from sklearn.metrics import precision_score
precision_score(y_test, predictions, pos_label = "ALTO")

: 

In [ ]:
features_names = X.columns
features_names

: 

In [ ]:
#obtenemos aquellas variables que son más importantes en nuestro arbol de decisión 
clf.feature_importances_

: 

In [ ]:
features_dt = pd.DataFrame(clf.feature_importances_, index = features_names)
features_dt

: 

In [ ]:
features_dt.plot(kind="bar")

: 

In [ ]:
#Graficamos nuestro arbol de decision con una profundidad de 5 
#podemos concluir que una mayor acidez y una menor acidez volatil son caracteristicas imporantes para un buen vino
from sklearn import tree 
from matplotlib import pyplot as plt

fig = plt.figure(figsize=(20,18)) 
_ = tree.plot_tree(clf,
               feature_names = features_names,
               class_names = {0:"BAJO",1:"ALTO"},
               filled = True,
               fontsize = 12)

: 